# Using PhysiCOOL with third-party libraries

The `BlackBoxModel` class can be used in combination with other libraries to perform parameter studies and optimization routines that have still not been implemented in PhysiCOOL.

Here, we describe how to couple our approach with the [psweep](https://github.com/elcorto/psweep) library, which can be used to set up and run parameter studies. The PhysiCell project used is the one described in the motility example. 

A Pandas DataFrame is created by `psweep` with the model outputs for each parameter set and some additional information (run_id, run_time...). Results are also stored in a folder called `calc`.

This approach can be used with other libraries, as users see fit.

In [1]:
from physicool. optimization import PhysiCellBlackBox
from physicool.updaters import CellUpdater, update_motility_values
from physicool.processing import get_final_y_position

import psweep as ps
import shutil

# Compiles the project and creates a black box object for it
# opt.compile_project()    
black_box = PhysiCellBlackBox()

# Define the updater we want to use (change motility data)
updater = CellUpdater(updater_function=update_motility_values,
                     config_path="config/PhysiCell_settings.xml", 
                     cell_definition_name="default")

# Assign the updater and processor to the black box
black_box.updater = updater
black_box.processor = get_final_y_position
black_box.version = "1.9.1"

def func(pset): 
    """
    Runs the black box model with the values selected by psweep.
    """
    shutil.rmtree("temp", ignore_errors=True)
    metric = black_box.run(pset)
    return {'result': metric}

# Choose parameters
# (Creates a grid [["speed": 1.0, "bias": 0.2],
#                  ["speed": 1.0, "bias": 0.6]])
a = ps.plist('migration_bias', [0.2, 0.6])
b = ps.plist('speed', [1.0])
params = ps.pgrid(a,b)

# Get a DataFrame with all the simulation results
# psweep will run func() with each pair of the params grid
df = ps.run_local(func, params)
df.head()

,migration_bias,speed,_pset_hash,_run_id,_pset_id,_calc_dir,_time_utc,_pset_seq,_run_seq,result,_pset_runtime
0,0.2,1.0,cd3eeb3b2452a69eac62b2d2d76d9f29c1d19be4,4d0aeab4-c3e8-4548-884d-dbf6f9353aec,a50e62b1-c463-4450-a4ba-709adbabb29f,calc,2023-01-29 02:19:09.673277855,0,0,"[-178.70691472781832, -188.68949563900048, -29...",2.644449
1,0.6,1.0,5a9a45aab62d3a8fed2acdc73b803e96429a24f4,4d0aeab4-c3e8-4548-884d-dbf6f9353aec,b4ef93bc-eb22-4c37-807f-1416ca0c439e,calc,2023-01-29 02:19:12.321578026,1,0,"[-450.0, -190.44424389791126, -403.90800062148...",2.505751
2,0.2,1.0,cd3eeb3b2452a69eac62b2d2d76d9f29c1d19be4,7a39aceb-294b-44c5-81dd-80aac1025d48,9a63fcc5-1e2d-42fa-9726-3ba5ac1a6f9f,calc,2023-01-29 02:19:21.191598177,2,1,"[-450.0, -152.29380180446478, -367.02854609208...",2.547715
3,0.6,1.0,5a9a45aab62d3a8fed2acdc73b803e96429a24f4,7a39aceb-294b-44c5-81dd-80aac1025d48,0093182a-9644-4ca5-8a6a-efda2c24654e,calc,2023-01-29 02:19:23.742476940,3,1,"[-229.5916817047852, -317.05699368798525, -183...",2.620152
